# Vector Stores

## Confirming that libraries are installed

## Importing libraries and API key

In [ ]:

import chromadb
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader



## Loading and splitting the documents

In [ ]:
loader = TextLoader("FDR_State_of_Union_1944.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

## Creating the embeddings

In [ ]:
embedding_function = OpenAIEmbeddings()

## Creating the database

In [ ]:
db = Chroma.from_documents(docs, embedding_function, persist_directory="./speech_new_db")

In [ ]:
db.persist()

In [ ]:
db_new_connection = Chroma(persist_directory='./speech_new_db', embedding_function=embedding_function)
# necessary if you restart the computer

## Doing a similarity search

In [ ]:
new_doc = "What did FDR say about the cost of food law?"
# All Chroma will do is return which document has the most similar language to this prompt. You could just write "cost of food law, FDR" and it would probably get the same result.

In [ ]:
similar_docs = db.similarity_search(new_doc)

In [ ]:
print(similar_docs[0].page_content)
# by default it returns the four most similar documents, going from most to least similar

## Loading another document into the database

In [ ]:
loader = TextLoader("Lincoln_State_of_Union_1862.txt")
documents = loader.load()

In [ ]:
docs = text_splitter.split_documents(documents)
# the warnings are because it's shorter and harder to make reasonable chunks of exactly 500

In [ ]:
db = Chroma.from_documents(docs, embedding_function, persist_directory="./speech_new_db")

In [ ]:
docs = db.similarity_search("slavery")

In [ ]:
print(docs[0].page_content)

In [ ]:
docs = db.similarity_search("cost of food law")

In [ ]:
print(docs[0].page_content)
# by this you can check that the database has borth speeches in it

In [ ]:
print(docs[0].metadata)
# this shows which source it's from

## Using a vector store retriever

In [ ]:
retriever = db.as_retriever()
# this retriever object will be the basis of multi query retrieval and context compression

In [ ]:
results = retriever.get_relevant_documents("cost of food law")

In [ ]:
print(results[0].page_content)